# Label side

In [4]:
import numpy as np
from jesse import helpers, research

_, candles = research.get_candles(
    "Binance Perpetual Futures",
    "BTC-USDT",
    "1m",
    helpers.date_to_timestamp("2022-07-01"),
    helpers.date_to_timestamp("2025-08-25"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)

candles.shape

(1657440, 6)

In [5]:
np.save("data/btc_1m.npy", candles)
del candles

In [6]:
import numpy as np

candles = np.load("data/btc_1m.npy")
candles.shape

(1657440, 6)

In [7]:
from src.bars.fusion.deap_v1 import DeapBarV1

bar_container = DeapBarV1(max_bars=500000)
bar_container.update_with_candles(candles)
merged_bar = bar_container.get_fusion_bars()
merged_bar.shape

(5789, 6)

In [8]:
np.save("data/bar_deap_v1.npy", merged_bar)

In [9]:
from research.labeler.gmm_labeler import GMMLabeler

labeler_L4 = GMMLabeler(merged_bar, 4)
labeler_L4.plot_label_on_candles()

[I 2025-09-08 10:30:38,840] A new study created in memory with name: no-name-43af4abd-db30-45d7-b5a4-aa46e5e38cb2
[I 2025-09-08 10:30:39,546] Trial 0 finished with value: 24.328785532080833 and parameters: {'random_state': 194}. Best is trial 0 with value: 24.328785532080833.
[I 2025-09-08 10:30:40,294] Trial 1 finished with value: 1.7704358515294392 and parameters: {'random_state': 524}. Best is trial 0 with value: 24.328785532080833.
[I 2025-09-08 10:30:41,093] Trial 2 finished with value: 1.770435851529439 and parameters: {'random_state': 384}. Best is trial 0 with value: 24.328785532080833.
[I 2025-09-08 10:30:41,661] Trial 3 finished with value: 24.328785532080833 and parameters: {'random_state': 312}. Best is trial 0 with value: 24.328785532080833.
[I 2025-09-08 10:30:42,188] Trial 4 finished with value: 24.328785532080833 and parameters: {'random_state': 735}. Best is trial 0 with value: 24.328785532080833.
[I 2025-09-08 10:30:42,773] Trial 5 finished with value: 24.328785532080

In [11]:
# labeler_L4.plot_label_returns()

In [10]:
labeler_L5 = GMMLabeler(merged_bar, 5)
labeler_L5.plot_label_on_candles()

[I 2025-09-08 10:30:52,762] A new study created in memory with name: no-name-aec3b150-cedb-435e-adb3-5aada2bc4951
[I 2025-09-08 10:30:53,254] Trial 0 finished with value: 26.52327201119823 and parameters: {'random_state': 970}. Best is trial 0 with value: 26.52327201119823.
[I 2025-09-08 10:30:53,878] Trial 1 finished with value: 21.672070756241887 and parameters: {'random_state': 674}. Best is trial 0 with value: 26.52327201119823.
[I 2025-09-08 10:30:54,613] Trial 2 finished with value: 21.672070756241887 and parameters: {'random_state': 961}. Best is trial 0 with value: 26.52327201119823.
[I 2025-09-08 10:30:55,267] Trial 3 finished with value: 21.672070756241887 and parameters: {'random_state': 356}. Best is trial 0 with value: 26.52327201119823.
[I 2025-09-08 10:30:55,802] Trial 4 finished with value: 1.7623536344764887 and parameters: {'random_state': 295}. Best is trial 0 with value: 26.52327201119823.
[I 2025-09-08 10:30:56,417] Trial 5 finished with value: 21.672070756241887 a

In [11]:
labeler_L6 = GMMLabeler(merged_bar, 6)
labeler_L6.plot_label_on_candles()

[I 2025-09-08 10:31:03,874] A new study created in memory with name: no-name-a9038ee8-615e-4f3a-af5c-785769082bcf
[I 2025-09-08 10:31:04,340] Trial 0 finished with value: 17.210960052163223 and parameters: {'random_state': 549}. Best is trial 0 with value: 17.210960052163223.
[I 2025-09-08 10:31:05,153] Trial 1 finished with value: 21.783773068854025 and parameters: {'random_state': 314}. Best is trial 1 with value: 21.783773068854025.
[I 2025-09-08 10:31:05,988] Trial 2 finished with value: 18.73551473510547 and parameters: {'random_state': 919}. Best is trial 1 with value: 21.783773068854025.
[I 2025-09-08 10:31:06,690] Trial 3 finished with value: 1.7657961912320717 and parameters: {'random_state': 489}. Best is trial 1 with value: 21.783773068854025.
[I 2025-09-08 10:31:07,478] Trial 4 finished with value: 19.92492210012281 and parameters: {'random_state': 780}. Best is trial 1 with value: 21.783773068854025.
[I 2025-09-08 10:31:08,189] Trial 5 finished with value: 1.76579619123207

In [12]:
for labeler, idx in zip([labeler_L4, labeler_L5, labeler_L6], [4, 5, 6]):
    label_hard = labeler.label_hard_state
    print(f"{label_hard.shape = }")
    np.save(f"data/label_hard_L{idx}.npy", label_hard)
    label_direction_force = labeler.label_direction_force
    print(f"{label_direction_force.shape = }")
    np.save(f"data/label_direction_L{idx}.npy", label_direction_force)

label_hard.shape = (5785,)
label_direction_force.shape = (5785,)
label_hard.shape = (5784,)
label_direction_force.shape = (5784,)
label_hard.shape = (5783,)
label_direction_force.shape = (5783,)


# Label meta

In [1]:
import numpy as np
import pandas as pd
from strategies.BinanceBtcEntropyBarV1.config import SIDE
from strategies.BinanceBtcEntropyBarV1.config import get_side_model

side_model = get_side_model(False)

merged_bar = np.load("data/merged_bar.npy")

df_features = pd.read_parquet("data/features.parquet")

side_res = side_model.predict(df_features[SIDE])

side_pred_label = np.where(side_res > 0.5, 1, -1)
print(side_pred_label.shape)

print(np.unique(side_pred_label, return_counts=True))

(13219,)
(array([-1,  1]), array([5080, 8139]))


In [2]:
close_prices = merged_bar[:, 2]
len_gap = len(close_prices) - len(side_pred_label)
close_prices = close_prices[len_gap - 1 : -1]

assert len(close_prices) == len(side_pred_label)

In [3]:
meta_label = np.zeros(len(side_pred_label))

TRADE_FEE = 0.05 / 100

start_idx = 0
cumsum_ret = 0
start_price = 0
for idx, (i, p) in enumerate(zip(side_pred_label, close_prices)):
    if i == 1 or i == -1:
        if idx == 0:
            # 开始持仓
            start_idx = idx
            start_price = p
            cumsum_ret -= TRADE_FEE
        elif side_pred_label[idx - 1] != i:
            # 反向持仓，先结算收益
            cumsum_ret -= TRADE_FEE
            cumsum_ret += np.log(p / start_price) * side_pred_label[idx - 1]
            if cumsum_ret > 0:
                meta_label[start_idx:idx] = 1
            cumsum_ret = 0
            start_price = p
            start_idx = idx
            cumsum_ret -= TRADE_FEE
        else:
            # 继续持仓
            continue
    else:
        raise ValueError(f"side_pred_label[{idx}] = {i} is not valid")
else:
    last_price = merged_bar[-1, 2]
    # 结算最后一根bar的持仓, 可能还没有结算，所以先不加trade fee
    if i == side_pred_label[idx - 1]:
        # 已经开仓，结算
        cumsum_ret += np.log(last_price / start_price) * side_pred_label[idx - 1]
        if cumsum_ret > 0:
            meta_label[start_idx:] = 1
    else:
        # 反向开仓
        cumsum_ret -= TRADE_FEE
        cumsum_ret += np.log(last_price / start_price) * side_pred_label[idx - 1]
        if cumsum_ret > 0:
            meta_label[start_idx:] = 1


print(np.unique(meta_label, return_counts=True))

(array([0., 1.]), array([  669, 12550]))


In [4]:
np.save("data/label_meta.npy", meta_label)